### Part Segmentation
⚓ https://github.com/soumik12345/point-cloud-segmentation 

### Downloads & Imports

In [1]:
!git clone https://github.com/soumik12345/point-cloud-segmentation -q
!pip install wandb ml_collections -qqq

     |████████████████████████████████| 1.9 MB 14.7 MB/s 
     |████████████████████████████████| 77 kB 6.7 MB/s 
     |████████████████████████████████| 166 kB 72.8 MB/s 
     |████████████████████████████████| 182 kB 78.8 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 166 kB 64.6 MB/s 
     |████████████████████████████████| 162 kB 71.7 MB/s 
     |████████████████████████████████| 162 kB 58.2 MB/s 
     |████████████████████████████████| 158 kB 64.3 MB/s 
     |████████████████████████████████| 157 kB 25.1 MB/s 
     |████████████████████████████████| 157 kB 62.9 MB/s 
     |████████████████████████████████| 157 kB 64.8 MB/s 
     |████████████████████████████████| 157 kB 73.7 MB/s 
     |████████████████████████████████| 157 kB 69.3 MB/s 
     |████████████████████████████████| 157 kB 65.8 MB/s 
     |████████████████████████████████| 157 kB 70.3 MB/s 
     |████████████████████████████████| 156 kB 10.5 MB/s 


In [ ]:
import sys

sys.path.append("point-cloud-segmentation")

from point_seg import transform_block
from point_seg import ShapeNetCoreLoaderInMemory
from configs import shapenetcore

import json
import numpy as np
import pandas as pd
import tensorflow as tf
import plotly.express as px

### Config

In [ ]:
CATEGORY_TO_MODEL = {
    "airplane": "https://github.com/soumik12345/point-cloud-segmentation/releases/download/v0.3/airplane.gz",
    "car": "https://github.com/soumik12345/point-cloud-segmentation/releases/download/v0.3/car.gz",
    "chair": "https://github.com/soumik12345/point-cloud-segmentation/releases/download/v0.3/chair.tar.gz",
    "table": "https://github.com/soumik12345/point-cloud-segmentation/releases/download/v0.3/table.tar.gz",
}


CATEGORY = "Table"  #@param ["Airplane", "Table", "Car", "Chair"] {type:"string"}
MODEL_URL = CATEGORY_TO_MODEL[CATEGORY.lower()]
CONFIGS = shapenetcore.get_config()
METADATA_URL = "https://github.com/soumik12345/point-cloud-segmentation/releases/download/v0.2/metadata.json"

### Load Metadata

In [ ]:
metadata_path = tf.keras.utils.get_file(
    origin=METADATA_URL, fname=METADATA_URL.split("/")[-1]
)
model_path = tf.keras.utils.get_file(
    origin=MODEL_URL, fname=MODEL_URL.split("/")[-1], untar=True
)

with open(metadata_path) as json_file:
    metadata = json.load(json_file)

LABELS = metadata[CATEGORY]["lables"]
COLORS = metadata[CATEGORY]["colors"]

### Visualization utils

In [ ]:
def visualize_data(point_cloud, labels):
    fig = px.scatter_3d(
        pd.DataFrame(
            data={
                "x": point_cloud[:, 0],
                "y": point_cloud[:, 1],
                "z": point_cloud[:, 2],
                "label": labels,
            }
        ),
        x="x",
        y="y",
        z="z",
        color="label",
        labels={"label": "Label"},
        color_discrete_sequence=COLORS,
        category_orders={"label": LABELS},
    )
    fig.show()


def visualize_single_point_cloud(point_clouds, label_clouds, idx):
    label_map = LABELS + ["none"]
    point_cloud = point_clouds[idx]
    label_cloud = label_clouds[idx]
    visualize_data(point_cloud, [label_map[np.argmax(label)] for label in label_cloud])

In [ ]:
data_loader = ShapeNetCoreLoaderInMemory(
    object_category=CATEGORY, n_sampled_points=CONFIGS.num_points,
)
data_loader.load_data()
_, val_dataset = data_loader.get_datasets(
    val_split=CONFIGS.val_split, batch_size=CONFIGS.batch_size,
)

100%|██████████| 7750/7750 [00:22<00:00, 342.54it/s]


### Load model for inference

In [ ]:
segmentation_model = tf.keras.models.load_model(
    model_path,
    custom_objects={"OrthogonalRegularizer": transform_block.OrthogonalRegularizer},
)
val_data_batch = next(iter(val_dataset))
val_predictions = segmentation_model.predict(val_data_batch[0])

1/1 [==============================] - 4s 4s/step


In [ ]:
idx = np.random.choice(len(val_data_batch[0]))
print(f"Index selected: {idx}")

# Plotting with ground-truth.
print("***********Ground-truth***********")
visualize_single_point_cloud(val_data_batch[0], val_data_batch[1], idx)

# Plotting with predicted labels.
print("***********Predicted***********")
visualize_single_point_cloud(val_data_batch[0], val_predictions, idx)

Index selected: 13
***********Ground-truth***********


***********Predicted***********
